# ASLI

In [ ]:
import pandas as pd
import numpy as np


from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
import warnings

warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_excel('/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA/فولاد.xlsx')
# Function to classify return
def classify_return(return_value):
    if return_value > 5:
        return 'Above +5'
    elif 2.5 < return_value <= 5:
        return '+2.5 to +5'
    elif 0 <= return_value <= 2.5:
        return '+0 to +2.5'
    elif -5 <= return_value < 0:
        return '0 to -5'
    else:
        return 'Below -5'

# RANDOM FOREST Regressor

In [ ]:


from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import warnings

warnings.filterwarnings('ignore')


# Define the preprocessing steps
scaling = StandardScaler()



# Define the model
model = RandomForestRegressor()

# Define the pipeline
pipeline = make_pipeline(
    scaling,
    model
)

# Define the function to apply to each group
def process_group(group):
    y = group['Return_Class']
    X = group.select_dtypes(include='number').drop(columns=['Return_Class'])

    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Fit the RandomizedSearchCV object
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    mae = mean_absolute_error(y_true=y_test, y_pred=y_pred)
    mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)

    # Retrieve feature importances from the best estimator
    feature_importance = pipeline.named_steps['randomforestregressor'].feature_importances_

    # Sort features by importance
    sorted_indices = np.argsort(feature_importance)[::-1]
    cumulative_importance = 0.0
    selected_features = []
    for idx in sorted_indices:
        cumulative_importance += feature_importance[idx]
        selected_features.append(X.columns[idx])
        if cumulative_importance >= 0.8:
            break

    return mae, mse, rmse, r2, selected_features


data = pd.read_excel('/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA/فولاد.xlsx')
# Function to classify return
def classify_return(return_value):
    if return_value > 5:
        return 'Above +5'
    elif 2.5 < return_value <= 5:
        return '+2.5 to +5'
    elif 0 <= return_value <= 2.5:
        return '+0 to +2.5'
    elif -5 <= return_value < 0:
        return '0 to -5'
    else:
        return 'Below -5'

# Group by year and month, then iterate through the groups
result = {}
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for (year, month), group in data.groupby([data['Date'].dt.year, data['Date'].dt.month]):
    # Filter the DataFrame for the data of the current month and all previous months
    filtered_data = data[(data['Date'].dt.year < year) | ((data['Date'].dt.year == year) & (data['Date'].dt.month <= month))]

    df = pd.DataFrame(filtered_data)
    Return_Class = df['پایانی*سهم*'].pct_change() * 100
    df['Return_Class'] = Return_Class.apply(classify_return)

    df['Return_Class'] = label_encoder.fit_transform(df['Return_Class'])
    df = df.dropna(subset=['Return_Class'])
    print(f' working on {year}-{month} ...') 
    scores = process_group(df)

    mae, mse, rmse, r2, selected_features = scores

    print(f'{year}-{month}') 
    print(f'mae = {mae}')
    print(f'mse = {mse}')
    print(f'rmse = {rmse}')
    print(f'r2 = {r2}')
    print(f'selected_features = {selected_features}')






    

In [ ]:
rf_result = pd.DataFrame(result, index=['mae','mse','rmse','r2','selected_features'])

In [ ]:
rf_result

In [ ]:
print(f' + : {len(rf_result.iloc[3][rf_result.iloc[3] > 0])}')
print(f' - : {len(rf_result.iloc[3][rf_result.iloc[3] < 0])}')
print(f' 0 : {len(rf_result.iloc[3][rf_result.iloc[3] == 0])}')

In [ ]:
mean_score = rf_result.iloc[3][rf_result.iloc[3] > 0].mean()
mean_score

# Rigde regrresion

In [ ]:


from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
import warnings

'''
param_dist = {
    'randomforestregressor__n_estimators': [100, 200, 300],
    'randomforestregressor__max_depth': [None, 5, 10, 15],
    'randomforestregressor__min_samples_split': [2, 5, 10],
    'randomforestregressor__min_samples_leaf': [1, 2, 4],
    'randomforestregressor__max_features': ['auto', 'sqrt', 'log2'],
    'randomforestregressor__bootstrap': [True, False]
}
'''
warnings.filterwarnings('ignore')


# Define the preprocessing steps
scaling = StandardScaler()



# Define the model
model = Ridge()

# Define the pipeline
pipeline = make_pipeline(
    scaling,
    model
)
'''
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    n_jobs=-1
)
'''
# Define the function to apply to each group
def process_group(group):
    y = group['Return_Class']
    X = group.select_dtypes(include='number').drop(columns=['Return_Class'])

    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Fit the RandomizedSearchCV object
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    mae = mean_absolute_error(y_true=y_test, y_pred=y_pred)
    mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)

    # Retrieve feature importances from the best estimator
    feature_importance = pipeline.named_steps['ridge'].coef_

    # Sort features by importance
    sorted_indices = np.argsort(feature_importance)[::-1]
    cumulative_importance = 0.0
    selected_features = []
    for idx in sorted_indices:
        cumulative_importance += feature_importance[idx]
        selected_features.append(X.columns[idx])
        if cumulative_importance >= 0.8:
            break

    return mae, mse, rmse, r2, selected_features

# Apply the function to each group
grouped_data = data.groupby('Year_Month')
scores = grouped_data.apply(process_group)
result = {}

# Iterate over index and values of the Series
for month, (mae, mse, rmse, r2, selected_features) in scores.items():
    result[f'{month}'] = (mae, mse, rmse, r2, selected_features)

In [ ]:
r2_result = pd.DataFrame(result, index=['mae','mse','rmse','r2','selected_features'])
r2_result

In [ ]:
print(f' + : {len(r2_result.iloc[3][r2_result.iloc[3] > 0])}')
print(f' - : {len(r2_result.iloc[3][r2_result.iloc[3] < 0])}')
print(f' 0 : {len(r2_result.iloc[3][r2_result.iloc[3] == 0])}')

# SVM model

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Define the preprocessing steps
scaling = StandardScaler()

# Define the model
model = SVR(kernel='rbf')  # Using Radial Basis Function kernel as an example

# Define the pipeline
pipeline = make_pipeline(
    scaling,
    model
)

# Define the function to apply to each group
def process_group(group):
    y = group['Return_Class']
    X = group.select_dtypes(include='number').drop(columns=['Return_Class'])

    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Fit the model
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    mae = mean_absolute_error(y_true=y_test, y_pred=y_pred)
    mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)

    # Retrieve feature importances from the SVR model
    # Since SVR doesn't have inherent feature_importances_, we can use coefficients for linear kernel
    if isinstance(model, SVR) and model.kernel == 'linear':
        feature_importance = np.abs(model.coef_[0])
    else:
        # For non-linear kernels or if the model is not linear, extracting feature importance is complex
        # Consider other methods like permutation importance or SHAP values for non-linear SVR models
        feature_importance = None

    selected_features = X.columns.tolist()  # Placeholder for feature importance extraction for non-linear kernels

    return mae, mse, rmse, r2, selected_features, feature_importance

# Apply the function to each group
grouped_data = data.groupby('Year_Month')
scores = grouped_data.apply(process_group)
result = {}

# Iterate over index and values of the Series
for month, (mae, mse, rmse, r2, selected_features, feature_importance) in scores.items():
    result[f'{month}'] = (mae, mse, rmse, r2, selected_features, feature_importance)


In [ ]:
svm_result = pd.DataFrame(result, index=['mae','mse','rmse','r2','selected_features','feature_importance'])

In [ ]:
svm_result

In [ ]:
t = pd.DataFrame(pd.date_range(start='2013-09-01', end='2023-08-31', freq='D'), columns=['Date'])

In [ ]:
t['Year_Month'] = t.Date.dt.to_period('M')

In [ ]:

grouped_data = t.groupby('Year_Month')

for i in range(len(grouped_data)):
    for month, group in grouped_data:
        print(month[:i+1])
        print('-------------')
        break
    

In [ ]:
l = ('A', 'B', 'C')
for i in range(len(l)):
    print(l[:i+1])
    print('-------------')

In [ ]:
accumulated_data = []
for _, data in grouped_data:
    print('--------------')
    print(data['Year_Month'].iloc[-1])
    print('--------------')
    accumulated_data.append(data)  # Append data for each group
    if len(accumulated_data) > 1:
        accumulated_data[-1] = pd.concat(accumulated_data[-2:], axis=0)  # Accumulate data
    


In [ ]:
accumulated_data

In [ ]:


# Group by year and month, then iterate through the groups
for (year, month), group in data.groupby([data['Date'].dt.year, data['Date'].dt.month]):
    # Filter the DataFrame for the data of the current month and all previous months
    filtered_data = data[(data['Date'].dt.year < year) | ((data['Date'].dt.year == year) & (data['Date'].dt.month <= month))]
    
    # Use 'filtered_data' for your analysis or processing
    print(f"Data up to {year}-{month}:")
    print(filtered_data)




In [ ]:
import pandas as pd
import numpy as np


from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
import warnings

warnings.filterwarnings('ignore')
data = pd.read_excel('/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA/فولاد.xlsx')
# Function to classify return
def classify_return(return_value):
    if return_value > 5:
        return 'Above +5'
    elif 2.5 < return_value <= 5:
        return '+2.5 to +5'
    elif 0 <= return_value <= 2.5:
        return '+0 to +2.5'
    elif -5 <= return_value < 0:
        return '0 to -5'
    else:
        return 'Below -5'

# Weighted Model

In [ ]:


# Define the preprocessing steps
scaling = StandardScaler()

# Define the model
model = RandomForestClassifier()

# Define the pipeline
pipeline = make_pipeline(
    scaling,
    model
)

# Define the function to apply to each group
def process_group(old_data, new_data):
    y_old = old_data['Return_Class']
    X_old = old_data.select_dtypes(include='number').drop(columns=['Return_Class'])

    y_new = new_data['Return_Class']
    X_new = new_data.select_dtypes(include='number').drop(columns=['Return_Class'])

    # Train-test split for old and new data separately
    X_old_train, X_old_test, y_old_train, y_old_test = train_test_split(X_old, y_old, test_size=0.2)
    X_new_train, X_new_test, y_new_train, y_new_test = train_test_split(X_new, y_new, test_size=0.2)

    # Fit the model on a combination of old and new data with 50% weightage
    X_train_combined = pd.concat([X_old_train, X_new_train])
    y_train_combined = pd.concat([y_old_train, y_new_train])
    sample_weights = [0.5] * len(X_old_train) + [0.5] * len(X_new_train)  # 50% weight for old and new

    # Fit the model
    pipeline.fit(X_train_combined, y_train_combined, randomforestclassifier__sample_weight=sample_weights)
    # Preparing prediction part
    X_test_combiend = pd.concat([X_old_test, X_new_test])
    y_test_combined = pd.concat([y_old_test, y_new_test])

    # Predict on the test set (either old or new)
    y_pred = pipeline.predict(X_test_combiend)

    mae = mean_absolute_error(y_true=y_test_combined, y_pred=y_pred)
    mse = mean_squared_error(y_true=y_test_combined, y_pred=y_pred)
    rmse = mean_squared_error(y_test_combined, y_pred, squared=False)
    r2 = r2_score(y_test_combined, y_pred)

    # Retrieve feature importances from the best estimator
    feature_importance = pipeline.named_steps['randomforestclassifier'].feature_importances_

    # Sort features by importance
    sorted_indices = np.argsort(feature_importance)[::-1]
    cumulative_importance = 0.0
    selected_features = []
    for idx in sorted_indices:
        cumulative_importance += feature_importance[idx]
        selected_features.append(X_old.columns[idx])
        if cumulative_importance >= 0.8:
            break

    return mae, mse, rmse, r2, selected_features


data = pd.read_excel('/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA/فولاد.xlsx')
n = 0

# Function to classify return
def classify_return(return_value):
    if return_value > 5:
        return 'Above +5'
    elif 2.5 < return_value <= 5:
        return '+2.5 to +5'
    elif 0 <= return_value <= 2.5:
        return '+0 to +2.5'
    elif -5 <= return_value < 0:
        return '0 to -5'
    else:
        return 'Below -5'

# Initialize old_data to an empty dataframe
old_data = pd.DataFrame()

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

for (year, month), new_data in data.groupby([data['Date'].dt.year, data['Date'].dt.month]):
    if n == 0 :
        # Filter the DataFrame for the data of the current month and all previous months
        filtered_data = data[(data['Date'].dt.year < year) | ((data['Date'].dt.year == year) & (data['Date'].dt.month <= month))]

        df = pd.DataFrame(filtered_data)
        Return_Class = df['پایانی*سهم*'].pct_change() * 100
        df['Return_Class'] = Return_Class.apply(classify_return)

        df['Return_Class'] = label_encoder.fit_transform(df['Return_Class'])
        df = df.dropna(subset=['Return_Class'])
        old_data = df
        n = n + 1

        print(f'working on {year}-{month} ...')
        scores = process_group(old_data, df)

        mae, mse, rmse, r2, selected_features = scores

        print(f'{year}-{month}')
        print(f'mae = {mae}')
        print(f'mse = {mse}')
        print(f'rmse = {rmse}')
        print(f'r2 = {r2}')
        print(f'selected_features = {selected_features}')

        # Update old_data with the current new_data for the next iteration
        old_data = df
    else: 
        # Filter the DataFrame for the data of the current month and all previous months
        filtered_data = data[(data['Date'].dt.year < year) | ((data['Date'].dt.year == year) & (data['Date'].dt.month <= month))]

        df = pd.DataFrame(filtered_data)
        Return_Class = df['پایانی*سهم*'].pct_change() * 100
        df['Return_Class'] = Return_Class.apply(classify_return)

        df['Return_Class'] = label_encoder.fit_transform(df['Return_Class'])
        df = df.dropna(subset=['Return_Class'])
        old_data = df
        n = n + 1

        print(f'working on {year}-{month} ...')
        scores = process_group(old_data, df)

        mae, mse, rmse, r2, selected_features = scores

        print(f'{year}-{month}')
        print(f'mae = {mae}')
        print(f'mse = {mse}')
        print(f'rmse = {rmse}')
        print(f'r2 = {r2}')
        print(f'selected_features = {selected_features}')

        # Update old_data with the current new_data for the next iteration
        old_data = pd.concat([old_data, df])


In [ ]:
import os

In [ ]:
folder_path = '/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA'

file_list = os.listdir(folder_path)

for i in file_list:
    print(i)

# Rigde regrresion

In [ ]:
import pandas as pd
import numpy as np


from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
import warnings

warnings.filterwarnings('ignore')
data = pd.read_excel('/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA/فولاد.xlsx')
# Function to classify return
def classify_return(return_value):
    if return_value > 5:
        return 'Above +5'
    elif 2.5 < return_value <= 5:
        return '+2.5 to +5'
    elif 0 <= return_value <= 2.5:
        return '+0 to +2.5'
    elif -5 <= return_value < 0:
        return '0 to -5'
    else:
        return 'Below -5'

In [ ]:
from sklearn.linear_model

In [ ]:


# Define the preprocessing steps
scaling = StandardScaler()

# Define the model
model = R()

# Define the pipeline
pipeline = make_pipeline(
    scaling,
    model
)

# Define the function to apply to each group
def process_group(old_data, new_data):
    y_old = old_data['Return_Class']
    X_old = old_data.select_dtypes(include='number').drop(columns=['Return_Class'])

    y_new = new_data['Return_Class']
    X_new = new_data.select_dtypes(include='number').drop(columns=['Return_Class'])

    # Train-test split for old and new data separately
    X_old_train, X_old_test, y_old_train, y_old_test = train_test_split(X_old, y_old, test_size=0.2)
    X_new_train, X_new_test, y_new_train, y_new_test = train_test_split(X_new, y_new, test_size=0.2)

    # Fit the model on a combination of old and new data with 50% weightage
    X_train_combined = pd.concat([X_old_train, X_new_train])
    y_train_combined = pd.concat([y_old_train, y_new_train])
    sample_weights = [0.5] * len(X_old_train) + [0.5] * len(X_new_train)  # 50% weight for old and new

    # Fit the model
    pipeline.fit(X_train_combined, y_train_combined, randomforestregressor__sample_weight=sample_weights)
    # Preparing prediction part
    X_test_combiend = pd.concat([X_old_test, X_new_test])
    y_test_combined = pd.concat([y_old_test, y_new_test])

    # Predict on the test set (either old or new)
    y_pred = pipeline.predict(X_test_combiend)

    mae = mean_absolute_error(y_true=y_test_combined, y_pred=y_pred)
    mse = mean_squared_error(y_true=y_test_combined, y_pred=y_pred)
    rmse = mean_squared_error(y_test_combined, y_pred, squared=False)
    r2 = r2_score(y_test_combined, y_pred)

    # Retrieve feature importances from the best estimator
    feature_importance = pipeline.named_steps['randomforestregressor'].feature_importances_

    # Sort features by importance
    sorted_indices = np.argsort(feature_importance)[::-1]
    cumulative_importance = 0.0
    selected_features = []
    for idx in sorted_indices:
        cumulative_importance += feature_importance[idx]
        selected_features.append(X_old.columns[idx])
        if cumulative_importance >= 0.8:
            break

    return mae, mse, rmse, r2, selected_features


data = pd.read_excel('/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA/فولاد.xlsx')
n = 0

# Function to classify return
def classify_return(return_value):
    if return_value > 5:
        return 'Above +5'
    elif 2.5 < return_value <= 5:
        return '+2.5 to +5'
    elif 0 <= return_value <= 2.5:
        return '+0 to +2.5'
    elif -5 <= return_value < 0:
        return '0 to -5'
    else:
        return 'Below -5'

# Initialize old_data to an empty dataframe
old_data = pd.DataFrame()

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

for (year, month), new_data in data.groupby([data['Date'].dt.year, data['Date'].dt.month]):
    if n == 0 :
        # Filter the DataFrame for the data of the current month and all previous months
        filtered_data = data[(data['Date'].dt.year < year) | ((data['Date'].dt.year == year) & (data['Date'].dt.month <= month))]

        df = pd.DataFrame(filtered_data)
        Return_Class = df['پایانی*سهم*'].pct_change() * 100
        df['Return_Class'] = Return_Class.apply(classify_return)

        df['Return_Class'] = label_encoder.fit_transform(df['Return_Class'])
        df = df.dropna(subset=['Return_Class'])
        old_data = df
        n = n + 1

        print(f'working on {year}-{month} ...')
        scores = process_group(old_data, df)

        mae, mse, rmse, r2, selected_features = scores

        print(f'{year}-{month}')
        print(f'mae = {mae}')
        print(f'mse = {mse}')
        print(f'rmse = {rmse}')
        print(f'r2 = {r2}')
        print(f'selected_features = {selected_features}')

        # Update old_data with the current new_data for the next iteration
        old_data = df
    else: 
        # Filter the DataFrame for the data of the current month and all previous months
        filtered_data = data[(data['Date'].dt.year < year) | ((data['Date'].dt.year == year) & (data['Date'].dt.month <= month))]

        df = pd.DataFrame(filtered_data)
        Return_Class = df['پایانی*سهم*'].pct_change() * 100
        df['Return_Class'] = Return_Class.apply(classify_return)

        df['Return_Class'] = label_encoder.fit_transform(df['Return_Class'])
        df = df.dropna(subset=['Return_Class'])
        old_data = df
        n = n + 1

        print(f'working on {year}-{month} ...')
        scores = process_group(old_data, df)

        mae, mse, rmse, r2, selected_features = scores

        print(f'{year}-{month}')
        print(f'mae = {mae}')
        print(f'mse = {mse}')
        print(f'rmse = {rmse}')
        print(f'r2 = {r2}')
        print(f'selected_features = {selected_features}')

        # Update old_data with the current new_data for the next iteration
        old_data = pd.concat([old_data, df])


In [ ]:
import pandas as pd
import numpy as np


from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
import warnings

warnings.filterwarnings('ignore')

# Function to classify return
def classify_return(return_value):
    if return_value > 5:
        return 'Above +5'
    elif 2.5 < return_value <= 5:
        return '+2.5 to +5'
    elif 0 <= return_value <= 2.5:
        return '+0 to +2.5'
    elif -5 <= return_value < 0:
        return '0 to -5'
    else:
        return 'Below -5'
# Weighted Model


# Define the preprocessing steps
scaling = StandardScaler()

# Define the model
model = RandomForestClassifier()

# Define the pipeline
pipeline = make_pipeline(
    scaling,
    model
)

# Define the function to apply to each group
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def process_group(old_data, new_data):
    y_old = old_data['Return_Class']
    X_old = old_data.select_dtypes(include='number').drop(columns=['Return_Class'])

    y_new = new_data['Return_Class']
    X_new = new_data.select_dtypes(include='number').drop(columns=['Return_Class'])

    # Train-test split for old and new data separately
    X_old_train, X_old_test, y_old_train, y_old_test = train_test_split(X_old, y_old, test_size=0.2)
    X_new_train, X_new_test, y_new_train, y_new_test = train_test_split(X_new, y_new, test_size=0.2)

    # Create a RandomForestClassifier
    classifier = RandomForestClassifier()

    # Fit the model on a combination of old and new data with 50% weightage
    X_train_combined = pd.concat([X_old_train, X_new_train])
    y_train_combined = pd.concat([y_old_train, y_new_train])
    sample_weights = [0.5] * len(X_old_train) + [0.5] * len(X_new_train)  # 50% weight for old and new

    # Fit the model
    classifier.fit(X_train_combined, y_train_combined, sample_weight=sample_weights)

    # Preparing prediction part
    X_test_combined = pd.concat([X_old_test, X_new_test])
    y_test_combined = pd.concat([y_old_test, y_new_test])

    # Predict on the test set (either old or new)
    y_pred = classifier.predict(X_test_combined)

    # Evaluation metrics for classification
    accuracy = accuracy_score(y_test_combined, y_pred)
    confusion = confusion_matrix(y_test_combined, y_pred)
    report = classification_report(y_test_combined, y_pred)

    # Retrieve feature importances from the classifier
    feature_importance = classifier.feature_importances_

    # Sort features by importance
    sorted_indices = np.argsort(feature_importance)[::-1]
    cumulative_importance = 0.0
    selected_features = []
    for idx in sorted_indices:
        cumulative_importance += feature_importance[idx]
        selected_features.append(X_old.columns[idx])
        if cumulative_importance >= 0.8:
            break

    return accuracy, confusion, report, selected_features

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Read data
data = pd.read_excel('/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA/فولاد.xlsx')

# Calculate percentage change and label Return_Class
data['Return_Class'] = data['پایانی*سهم*'].pct_change() * 100
data['Return_Class'] = data['Return_Class'].apply(classify_return)  # Assuming classify_return is defined

# Drop rows with NaN in 'Return_Class'
data = data.dropna(subset=['Return_Class'])



# Encode 'Return_Class'
label_encoder = LabelEncoder()
data['Return_Class'] = label_encoder.fit_transform(data['Return_Class'])

# Check if necessary columns are present
required_columns = ['Date', 'Return_Class']  # Add other required columns here
if not all(col in data.columns for col in required_columns):
    raise KeyError("Required columns are missing in the dataset.")

# Iterate through year-month groups
old_data = None
for (year, month), new_data in data.groupby([data['Date'].dt.year, data['Date'].dt.month]):
    # Filter the DataFrame for the data of the current month and all previous months
    filtered_data = data[(data['Date'].dt.year < year) | ((data['Date'].dt.year == year) & (data['Date'].dt.month <= month))]
    df = pd.DataFrame(filtered_data)

    # Confirm 'Return_Class' exists in df
    if 'Return_Class' not in df.columns:
        continue  # Skip iteration if 'Return_Class' is not present in the filtered DataFrame

    # Process data and get scores
    print(f'working on {year}-{month} ...')
    if old_data is None:
        old_data = pd.DataFrame()  # Reinitialize old_data if 'Return_Class' is not present
    scores = process_group(old_data, df)
    accuracy, confusion, report, selected_features = scores

    # Print scores
    print(f'{year}-{month}')
    print(f'accuracy = {accuracy}')
    print(f'confusion_matrix = {confusion}')
    print(f'report = {report}')
    print(f'selected Features = {selected_features}')

    # Update old_data for the next iteration
    old_data = pd.concat([old_data, df])

In [6]:
data = pd.read_excel('/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA/فولاد.xlsx')

In [7]:
data.columns

Index(['Date', 'تاریخ شمسی',
       'حجم معاملات اوراق با درآمد ثابت بلوکی-بورس*بورس*',
       'ارزش معاملات اوراق با درآمد ثابت بلوکی-بورس*بورس*',
       'قیمت به درآمد-بورس*بورس*', 'ارزش بازار (دلار آزاد)-بورس*بورس*',
       'درصد شناوری-بورس*بورس*', 'حجم معاملات سهام بلوکی-بورس*بورس*',
       'سود خالص-آخرین فصل-بورس*بورس*', 'قیمت به سود -بورس*بورس*',
       ...
       'Kurtosis', 'Skewness', 'ROC20', 'Volume Momentum 20', 'TEMA20',
       'Price Momentum 1M', 'PLRC12', 'Historical Volatility',
       'Realized_Volatility', 'ATR14'],
      dtype='object', length=144)

In [3]:
import os
import warnings

import numpy as np
import pandas as pd
from sklearn.model_selection import  train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.compose import ColumnTransformer #This modification separates the encoding step for categorical columns using LabelEncoder within the ColumnTransformer.

warnings.filterwarnings('ignore')

# Function to classify return
def classify_return(return_value):
    """
    Classify the given return value into different categories based on the value range.

    Parameters:
        return_value (float): The value to be classified.

    Returns:
        str: The category that the return value belongs to. Possible categories are:
             - 'Above +5' if the value is greater than 5
             - '+2.5 to +5' if the value is between 2.5 and 5 (inclusive)
             - '+0 to +2.5' if the value is between 0 and 2.5 (inclusive)
             - '0 to -5' if the value is between -5 and 0 (inclusive)
             - 'Below -5' if the value is less than -5
    """
    if return_value > 5:
        return 'Above +5'
    elif 2.5 < return_value <= 5:
        return '+2.5 to +5'
    elif 0 <= return_value <= 2.5:
        return '+0 to +2.5'
    elif -5 <= return_value < 0:
        return '0 to -5'
    else:
        return 'Below -5'


def process_group(old_data):

    label_encoder = LabelEncoder()
    Return_Class_old = old_data['پایانی*سهم*'].pct_change() * 100
    old_data['Return_Class'] = Return_Class_old.apply(classify_return)

    old_data['Return_Class'] = label_encoder.fit_transform(old_data['Return_Class'])
    old_data = old_data.dropna(subset=['Return_Class'])


    scaling = StandardScaler()
    # Select numeric columns except the target column
    numeric_columns = old_data.select_dtypes(include=[np.number]).drop(columns=['Return_Class']).columns.tolist()

    # Extract the subset of data with only numeric columns
    old_num_data = old_data[numeric_columns]



    y_old = old_data['Return_Class']
    X_old = scaling.fit_transform(old_num_data)





    # Train-test split for old and new data separately
    X_old_train, X_old_test, y_old_train, y_old_test = train_test_split(X_old, y_old, test_size=0.2)





    model = RandomForestClassifier()
    # Fit the model
    model.fit(X_old_train, y_old_train)




    # Predict on the test set (either old or new)
    y_pred = model.predict(X_old_test)

    # Evaluation metrics for classification
    accuracy = accuracy_score(y_old_test, y_pred)
    confusion = confusion_matrix(y_old_test, y_pred)
    report = classification_report(y_old_test, y_pred)

    # Retrieve feature importances from the classifier
    feature_importance = model.feature_importances_

    # Sort features by importance
    sorted_indices = np.argsort(feature_importance)[::-1]
    cumulative_importance = 0.0
    selected_features = []
    for idx in sorted_indices:
        cumulative_importance += feature_importance[idx]
        selected_features.append(numeric_columns[idx])
        if cumulative_importance >= 0.8:
            break

    return accuracy, confusion, report, selected_features


data = pd.read_excel('/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA/فولاد.xlsx')




old_data = None

for (year, month), new_data in data.groupby([data['Date'].dt.year, data['Date'].dt.month]):
    mask = (data['Date'].dt.year == year) & (data['Date'].dt.month == month)
    filtered_data = data[mask]
    now_df = pd.DataFrame(filtered_data)


    if old_data is None:
        old_data = now_df

    else:
        mask2 = ((data['Date'].dt.year == year) & (data['Date'].dt.month < month)) | ((data['Date'].dt.year < year))
        old_data_filtered = old_data[mask2]
        old_data = pd.concat([old_data_filtered, now_df])


    print(f'{year}-{month} ...')

    # Process old and new data
    scores = process_group(old_data.copy())  # Ensure copies to prevent data contamination

    accuracy, confusion, report, selected_features = scores

    print(f'{year}-{month}')
    print(f'accuracy = {accuracy}')
    print(f'confusion_matrix = {confusion}')
    print(f'report = {report}')
    print(f'selected Features = {selected_features}')



******* Now DATA Frame shape = (30, 144)*****
******* old DATA Frame shape = (30, 144)*****
2013-9 ...
2013-9
accuracy = 0.5
confusion_matrix = [[3 1 0]
 [1 0 0]
 [1 0 0]]
report =               precision    recall  f1-score   support

           0       0.60      0.75      0.67         4
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1

    accuracy                           0.50         6
   macro avg       0.20      0.25      0.22         6
weighted avg       0.40      0.50      0.44         6

selected Features = ['درصد تغییر*صنعت*', 'خرید حقیقی-حجم(تعداد سهام)*صنعت*', 'میزان تغییر*صنعت*', 'خرید حقوقی-حجم(تعداد سهام)*صنعت*', 'فروش حقیقی-ارزش(ریال)*صنعت*', 'حجم*صنعت*', 'سهام شناور*صنعت*', 'خالص خرید حقوقی ارزش-بورس*بورس*', 'فروش حقیقی-حجم(تعداد سهام)*صنعت*', 'شاخص آزاد شناور*کلان*', 'Gold Price', 'Stochastic Oscillator_slowd', 'ارزش معاملات*صنعت*', 'خرید حقیقی-ارزش(ریال)*صنعت*', 'P/B*صنعت*', 'فروش حقوقی حجم-بورس*بورس*', 'P/B

KeyboardInterrupt: 

In [ ]:
feature_selection_RF('/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA/فولاد.xlsx')

# ASLI ---> RF Regression

In [ ]:
import os
import warnings

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler

warnings.filterwarnings('ignore')

# Function to classify return
def classify_return(return_value):
    """
    Classify the given return value into different categories based on the value range.

    Parameters:
        return_value (float): The value to be classified.

    Returns:
        str: The category that the return value belongs to. Possible categories are:
             - 'Above +5' if the value is greater than 5
             - '+2.5 to +5' if the value is between 2.5 and 5 (inclusive)
             - '+0 to +2.5' if the value is between 0 and 2.5 (inclusive)
             - '0 to -5' if the value is between -5 and 0 (inclusive)
             - 'Below -5' if the value is less than -5
    """
    if return_value > 5:
        return 'Above +5'
    elif 2.5 < return_value <= 5:
        return '+2.5 to +5'
    elif 0 <= return_value <= 2.5:
        return '+0 to +2.5'
    elif -5 <= return_value < 0:
        return '0 to -5'
    else:
        return 'Below -5'

def feature_selection_RF(data_path):
        # *** Weighted Model 50/50 ***
    # Define the label encoder for the 'Return_Class' column
    label_encoder = LabelEncoder()

    # Define the preprocessing steps
    scaling = StandardScaler()

    # Define the model
    model = RandomForestRegressor()

    # Define the pipeline
    pipeline = make_pipeline(
        scaling,
        model
    )

    # Define the function to apply to each group
    def process_group(old_data, new_data):
        y_old = old_data['Return_Class']
        X_old = old_data.select_dtypes(include='number').drop(columns=['Return_Class'])

        y_new = new_data['Return_Class']
        X_new = new_data.select_dtypes(include='number').drop(columns=['Return_Class'])

        # Train-test split for old and new data separately
        X_old_train, X_old_test, y_old_train, y_old_test = train_test_split(X_old, y_old, test_size=0.2)
        X_new_train, X_new_test, y_new_train, y_new_test = train_test_split(X_new, y_new, test_size=0.2)

        # Fit the model on a combination of old and new data with 50% weightage
        X_train_combined = pd.concat([X_old_train, X_new_train])
        y_train_combined = pd.concat([y_old_train, y_new_train])
        sample_weights = [0.5] * len(X_old_train) + [0.5] * len(X_new_train)  # 50% weight for old and new

        # Fit the model
        pipeline.fit(X_train_combined, y_train_combined, randomforestregressor__sample_weight=sample_weights)
        # Preparing prediction part
        X_test_combiend = pd.concat([X_old_test, X_new_test])
        y_test_combined = pd.concat([y_old_test, y_new_test])

        # Predict on the test set (either old or new)
        y_pred = pipeline.predict(X_test_combiend)

        mae = mean_absolute_error(y_true=y_test_combined, y_pred=y_pred)
        mse = mean_squared_error(y_true=y_test_combined, y_pred=y_pred)
        rmse = mean_squared_error(y_test_combined, y_pred, squared=False)
        r2 = r2_score(y_test_combined, y_pred)

        # Retrieve feature importances from the best estimator
        feature_importance = pipeline.named_steps['randomforestregressor'].feature_importances_

        # Sort features by importance
        sorted_indices = np.argsort(feature_importance)[::-1]
        cumulative_importance = 0.0
        selected_features = []
        for idx in sorted_indices:
            cumulative_importance += feature_importance[idx]
            selected_features.append(X_old.columns[idx])
            if cumulative_importance >= 0.8:
                break

        return mae, mse, rmse, r2, selected_features


    data = pd.read_excel(data_path)
    n = 0

    # Initialize old_data to an empty dataframe
    old_data = pd.DataFrame()

    # Group the data by year and month


    final_result = {}

    for (year, month), new_data in data.groupby([data['Date'].dt.year, data['Date'].dt.month]):
        if n == 0 :
            # Filter the DataFrame for the data of the current month and all previous months
            filtered_data = data[(data['Date'].dt.year < year) | ((data['Date'].dt.year == year) & (data['Date'].dt.month <= month))]

            df = pd.DataFrame(filtered_data)
            Return_Class = df['پایانی*سهم*'].pct_change() * 100
            df['Return_Class'] = Return_Class.apply(classify_return)

            df['Return_Class'] = label_encoder.fit_transform(df['Return_Class'])
            df = df.dropna(subset=['Return_Class'])
            old_data = df
            n = n + 1

            print(f'{year}-{month} ...')
            scores = process_group(old_data, df)

            mae, mse, rmse, r2, selected_features = scores
            final_result[f'{year}-{month}'] = {
                'mae': mae,
                'mse': mse,
                'rmse': rmse,
                'r2': r2,
                'selected_features': selected_features
            }

            # Update old_data with the current new_data for the next iteration
            old_data = df
        else: 
            # Filter the DataFrame for the data of the current month and all previous months
            filtered_data = data[(data['Date'].dt.year < year) | ((data['Date'].dt.year == year) & (data['Date'].dt.month <= month))]

            df = pd.DataFrame(filtered_data)
            Return_Class = df['پایانی*سهم*'].pct_change() * 100
            df['Return_Class'] = Return_Class.apply(classify_return)

            df['Return_Class'] = label_encoder.fit_transform(df['Return_Class'])
            df = df.dropna(subset=['Return_Class'])
            old_data = df
            n = n + 1

            print(f'{year}-{month} ...')
            scores = process_group(old_data, df)

            mae, mse, rmse, r2, selected_features = scores

            final_result[f'{year}-{month}'] = {
                'mae': mae,
                'mse': mse,
                'rmse': rmse,
                'r2': r2,
                'selected_features': selected_features
            }

            # Update old_data with the current new_data for the next iteration
            old_data = pd.concat([old_data, df])


    df = pd.DataFrame(final_result)
    return df




folder_path = '/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA'
file_list = os.listdir(folder_path)
for file in file_list:
    file_path = f'{folder_path}/{file}'
    print(f'working on {file} ...')
    df = feature_selection_RF(file_path)
    df.to_excel(f'/home/amin/thesis/src/Step/Feature_selection_RF/{file}_RF', index=False)


# Corrected Code :

In [ ]:
import os
import warnings

import numpy as np
import pandas as pd
from sklearn.model_selection import  train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.compose import ColumnTransformer #This modification separates the encoding step for categorical columns using LabelEncoder within the ColumnTransformer.

warnings.filterwarnings('ignore')

# Function to classify return
def classify_return(return_value):
    """
    Classify the given return value into different categories based on the value range.

    Parameters:
        return_value (float): The value to be classified.

    Returns:
        str: The category that the return value belongs to. Possible categories are:
             - 'Above +5' if the value is greater than 5
             - '+2.5 to +5' if the value is between 2.5 and 5 (inclusive)
             - '+0 to +2.5' if the value is between 0 and 2.5 (inclusive)
             - '0 to -5' if the value is between -5 and 0 (inclusive)
             - 'Below -5' if the value is less than -5
    """
    if return_value > 5:
        return 'Above +5'
    elif 2.5 < return_value <= 5:
        return '+2.5 to +5'
    elif 0 <= return_value <= 2.5:
        return '+0 to +2.5'
    elif -5 <= return_value < 0:
        return '0 to -5'
    else:
        return 'Below -5'


def process_group(old_data, new_data):

    label_encoder = LabelEncoder()
    Return_Class_old = old_data['پایانی*سهم*'].pct_change() * 100
    old_data['Return_Class'] = Return_Class_old.apply(classify_return)

    old_data['Return_Class'] = label_encoder.fit_transform(old_data['Return_Class'])
    old_data = old_data.dropna(subset=['Return_Class'])

    Return_Class_new = new_data['پایانی*سهم*'].pct_change() * 100
    new_data['Return_Class'] = Return_Class_new.apply(classify_return)

    new_data['Return_Class'] = label_encoder.fit_transform(new_data['Return_Class'])
    new_data = new_data.dropna(subset=['Return_Class'])

    scaling = StandardScaler()
    # Select numeric columns except the target column
    numeric_columns = old_data.select_dtypes(include=[np.number]).drop(columns=['Return_Class']).columns.tolist()

    # Extract the subset of data with only numeric columns
    old_num_data = old_data[numeric_columns]

    

    y_old = old_data['Return_Class']
    X_old = scaling.fit_transform(old_num_data)


    # Select numeric columns except the target column
    numeric_columns = new_data.select_dtypes(include=[np.number]).drop(columns=['Return_Class']).columns.tolist()

    # Extract the subset of data with only numeric columns
    new_num_data = new_data[numeric_columns]


    y_new = new_data['Return_Class']
    X_new = scaling.fit_transform(new_num_data)


    # Train-test split for old and new data separately
    X_old_train, X_old_test, y_old_train, y_old_test = train_test_split(X_old, y_old, test_size=0.2)
    X_new_train, X_new_test, y_new_train, y_new_test = train_test_split(X_new, y_new, test_size=0.2)

    # Fit the model on a combination of old and new data with 50% weightage
    X_train_combined = pd.concat([pd.DataFrame(X_old_train), pd.DataFrame(X_new_train)])
    y_train_combined = pd.concat([pd.DataFrame(y_old_train), pd.DataFrame(y_new_train)])
    sample_weights = [0.5] * len(X_old_train) + [0.5] * len(X_new_train)  # 50% weight for old and new

    model = RandomForestClassifier()
    # Fit the model
    model.fit(X_train_combined, y_train_combined)


    # Preparing prediction part
    X_test_combined = pd.concat([pd.DataFrame(X_old_test), pd.DataFrame(X_new_test)])
    y_test_combined = pd.concat([pd.DataFrame(y_old_test), pd.DataFrame(y_new_test)])

    # Predict on the test set (either old or new)
    y_pred = model.predict(X_test_combined)

    # Evaluation metrics for classification
    accuracy = accuracy_score(y_test_combined, y_pred)
    confusion = confusion_matrix(y_test_combined, y_pred)
    report = classification_report(y_test_combined, y_pred)

    # Retrieve feature importances from the classifier
    feature_importance = model.feature_importances_

    # Sort features by importance
    sorted_indices = np.argsort(feature_importance)[::-1]
    cumulative_importance = 0.0
    selected_features = []
    for idx in sorted_indices:
        cumulative_importance += feature_importance[idx]
        selected_features.append(numeric_columns[idx])
        if cumulative_importance >= 0.8:
            break

    return accuracy, confusion, report, selected_features


data = pd.read_excel('/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA/فولاد.xlsx')




old_data = None

for (year, month), new_data in data.groupby([data['Date'].dt.year, data['Date'].dt.month]):
    mask = (data['Date'].dt.year == year) & (data['Date'].dt.month == month)
    filtered_data = data[mask]
    now_df = pd.DataFrame(filtered_data)
    print(f'******* Now DATA Frame shape = {now_df.shape}*****')

    if old_data is None:
        old_data = now_df
        print(f'******* old DATA Frame shape = {old_data.shape}*****')
    else:
        mask2 = ((data['Date'].dt.year == year) & (data['Date'].dt.month < month)) | ((data['Date'].dt.year < year))
        old_data_filtered = old_data[mask2]
        old_data = pd.concat([old_data_filtered, now_df])
        print(f'******* old DATA Frame shape = {old_data.shape}*****')

    print(f'{year}-{month} ...')

    # Process old and new data
    scores = process_group(old_data.copy(), now_df.copy())  # Ensure copies to prevent data contamination

    accuracy, confusion, report, selected_features = scores

    print(f'{year}-{month}')
    print(f'accuracy = {accuracy}')
    print(f'confusion_matrix = {confusion}')
    print(f'report = {report}')
    print(f'selected Features = {selected_features}')

